In [4]:
import cv2
import cvzone
import numpy as np
from ultralytics import YOLO
from paddleocr import PaddleOCR
from datetime import datetime

print("جميع المكتبات تم تثبيتها بنجاح!")


جميع المكتبات تم تثبيتها بنجاح!


cpu


In [6]:
########################################
################################


import cv2
import numpy as np
from ultralytics import YOLO
from paddleocr import PaddleOCR
from datetime import datetime

# Initialize PaddleOCR for license plate recognition
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Load YOLOv8 model for vehicle detection
model = YOLO('best.pt')

# Load the video
cap = cv2.VideoCapture('car.mp4')

# Define the region of interest (ROI) for the traffic light
traffic_light_roi = (900, 100, 960, 160)  # Example values

# To track recognized plates
processed_numbers = set()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for consistency
    frame = cv2.resize(frame, (1020, 500))

    # Detect vehicles using YOLOv8
    results = model(frame, stream=True)

    # Process results from YOLOv8
    for result in results:
        boxes = result.boxes  # Get the box object
        
        for box in boxes:  # Iterate over detected boxes
            # Get the bounding box details; box.xyxy contains x1, y1, x2, y2
            x1, y1, x2, y2 = box.xyxy[0]  # Unpack coordinates
            conf = box.conf[0]  # Get the confidence score
            cls = int(box.cls[0])  # Get the class ID
            
            # Check if the detected object is a vehicle (adjust class IDs as needed)
            if cls in [0, 2, 3]:  # Example class IDs for vehicles
                # Implement logic to check if vehicle is in violation
                traffic_light_status = "Red"  # Assuming red light for this example

                if traffic_light_status == "Red":
                    # Crop the detected region (license plate)
                    crop = frame[int(y1):int(y2), int(x1):int(x2)]

                    # Apply OCR to the cropped image
                    ocr_result = ocr.ocr(crop, cls=True)
                    if ocr_result and len(ocr_result) > 0:
                        text = ocr_result[0][0][1][0].strip()
                        print("ewwwwwwwwwww",text)

                        # Log recognized plate if valid and not processed before
                        if text and len(text) > 4 and text not in processed_numbers:
                            processed_numbers.add(text)
                            current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                            print(f"Recognized License Plate: {text} at {current_datetime}")

                            # Optionally, save to a file
                            with open("car_plate_data.txt", "a") as file:
                                file.write(f"{text}\t{current_datetime}\tRed light violation\n")

                    # Draw a rectangle around the detected license plate
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Draw the traffic light ROI
    cv2.rectangle(frame, (traffic_light_roi[0], traffic_light_roi[1]),
                  (traffic_light_roi[2], traffic_light_roi[3]), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Frame", frame)

    # Press 'Esc' key to exit
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()




[2024/10/01 05:14:35] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\makany/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\makany/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_nu

In [17]:
import cv2
import numpy as np
from ultralytics import YOLO
from paddleocr import PaddleOCR
from datetime import datetime

# Load the YOLO models
plate_model = YOLO("best.pt")  # Model for license plate detection
model = YOLO("yolov8n.pt")  # Model for general object detection

# Initialize PaddleOCR for license plate recognition
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Load the video
cap = cv2.VideoCapture('car.mp4')

# Define the region of interest (ROI) for the traffic light
traffic_light_roi = (900, 100, 960, 160)  # Example values

# To track recognized plates
processed_numbers = set()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for consistency
    frame = cv2.resize(frame, (1020, 500))

    # Detect objects (e.g., vehicles) using the general YOLO model
    results = model(frame, stream=True)

    # Process the detection results from the general YOLO model
    for result in results:
        boxes = result.boxes
        
        for box in boxes:
            # Get the bounding box details
            x1, y1, x2, y2 = box.xyxy[0]
            conf = box.conf[0]
            cls = int(box.cls[0])

            # Check if the detected object is a vehicle (adjust class IDs as needed)
            if cls in [0, 2, 3]:  # Example class IDs for vehicles
                traffic_light_status = "Red"  # Assuming red light for this example

                if traffic_light_status == "Red":
                    # Detect license plates on the cropped region using the plate_model
                    vehicle_crop = frame[int(y1):int(y2), int(x1):int(x2)]
                    plate_results = plate_model(vehicle_crop, stream=True)
                cv2.imshow("dsfsfsdf",vehicle_crop)
                    # Process license plate detection results
                    for plate_result in plate_results:
                        plate_boxes = plate_result.boxes

                        for plate_box in plate_boxes:
                            # Get license plate bounding box details
                            px1, py1, px2, py2 = plate_box.xyxy[0]

                            # Crop the detected region (license plate)
                            plate_crop = vehicle_crop[int(py1):int(py2), int(px1):int(px2)]

                            # Apply OCR to the cropped image
                            ocr_result = ocr.ocr(plate_crop, cls=True)
                            if ocr_result and len(ocr_result) > 0:
                                text = ocr_result[0][0][1][0].strip()

                                # Log recognized plate if valid and not processed before
                                if text and len(text) > 4 and text not in processed_numbers:
                                    processed_numbers.add(text)
                                    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                    print(f"Recognized License Plate: {text} at {current_datetime}")

                                    # Optionally, save to a file
                                    with open("car_plate_data.txt", "a") as file:
                                        file.write(f"{text}\t{current_datetime}\tRed light violation\n")

                            # Draw a rectangle around the detected license plate
                            cv2.rectangle(frame, (int(px1), int(py1)), (int(px2), int(py2)), (0, 255, 0), 2)

    # Draw the traffic light ROI
    cv2.rectangle(frame, (traffic_light_roi[0], traffic_light_roi[1]),
                  (traffic_light_roi[2], traffic_light_roi[3]), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Frame", frame)

    # Press 'Esc' key to exit
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()


IndentationError: unexpected indent (3851466684.py, line 54)

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
from paddleocr import PaddleOCR
from datetime import datetime

# Load the YOLO models
plate_model = YOLO("best.pt")  # Model for license plate detection
model = YOLO("yolov8n.pt")  # Model for general object detection

# Initialize PaddleOCR for license plate recognition
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Load the video
cap = cv2.VideoCapture('car.mp4')

# Define the region of interest (ROI) for the traffic light
traffic_light_roi = (900, 100, 960, 160)  # Example values

# To track recognized plates
processed_numbers = set()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for consistency
    frame = cv2.resize(frame, (1020, 500))

    # Detect objects (e.g., vehicles) using the general YOLO model
    results = model(frame, stream=True)

    # Process the detection results from the general YOLO model
    for result in results:
        boxes = result.boxes
        
        for box in boxes:
            # Get the bounding box details
            x1, y1, x2, y2 = box.xyxy[0]
            conf = box.conf[0]
            cls = int(box.cls[0])

            # Check if the detected object is a vehicle (adjust class IDs as needed)
            if cls in [0, 2, 3]:  # Example class IDs for vehicles
                traffic_light_status = "Red"  # Assuming red light for this example

                if traffic_light_status == "Red":
                    # Detect license plates on the cropped region using the plate_model
                    vehicle_crop = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # Display the cropped vehicle for visual verification
                    cv2.imshow("Vehicle Crop", vehicle_crop)

                    plate_results = plate_model(vehicle_crop, stream=True)

                    # Process license plate detection results
                    for plate_result in plate_results:
                        plate_boxes = plate_result.boxes

                        for plate_box in plate_boxes:
                            # Get license plate bounding box details
                            px1, py1, px2, py2 = plate_box.xyxy[0]

                            # Crop the detected region (license plate)
                            plate_crop = vehicle_crop[int(py1):int(py2), int(px1):int(px2)]

                            # Apply OCR to the cropped image
                            ocr_result = ocr.ocr(plate_crop, cls=True)
                            if ocr_result and len(ocr_result) > 0:
                                text = ocr_result[0][0][1][0].strip()

                                # Log recognized plate if valid and not processed before
                                if text and len(text) > 4 and text not in processed_numbers:
                                    processed_numbers.add(text)
                                    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                    print(f"Recognized License Plate: {text} at {current_datetime}")

                                    # Optionally, save to a file
                                    with open("car_plate_data.txt", "a") as file:
                                        file.write(f"{text}\t{current_datetime}\tRed light violation\n")

                            # Draw a rectangle around the detected license plate
                            cv2.rectangle(vehicle_crop, (int(px1), int(py1)), (int(px2), int(py2)), (0, 255, 0), 2)

    # Draw the traffic light ROI
    cv2.rectangle(frame, (traffic_light_roi[0], traffic_light_roi[1]),
                  (traffic_light_roi[2], traffic_light_roi[3]), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Frame", frame)

    # Press 'Esc' key to exit
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()

###############################


[2024/09/30 18:05:15] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\makany/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\makany/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=Non

TypeError: 'NoneType' object is not subscriptable

In [17]:
## Test##


import cvzone
import cv2
import numpy as np
from ultralytics import YOLO
from paddleocr import PaddleOCR
from datetime import datetime

# Load the YOLO models
plate_model = YOLO("best.pt")  # Model for license plate detection
model = YOLO("yolov8n.pt")  # Model for general object detection

# Initialize PaddleOCR for license plate recognition
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Load the video
cap = cv2.VideoCapture('car.mp4')

# Define the region of interest (ROI) for the traffic light
traffic_light_roi = (900, 100, 960, 160)  # Example values

# To track recognized plates
processed_numbers = set()
Traffic_color=""
po1_rec=(570 ,370)
po12_rec=(176 ,370)

width = 1020
height = 720

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for consistency
    frame = cv2.resize(frame, (1020, 720))

    # Detect objects (e.g., vehicles) using the general YOLO model
    results = model(frame, stream=True)
    
    black_image = np.zeros((height, width, 3), dtype=np.uint8)
    cv2.line(frame,(po1_rec),(po12_rec),(255, 0, 0),2)

    # Process the detection results from the general YOLO model
    for result in results:
        boxes = result.boxes
        
        for box in boxes:
            # Get the bounding box details
            x1, y1, x2, y2 = box.xyxy[0]
            
            x, y, w, z = int(x1), int(y1), int(x2), int(y2)
            w = w - x
            h = z - y
            cx = x + w // 2
            cy = y + h // 2
            conf = box.conf[0]
            cls = int(box.cls[0])

            if cls == 9:  # Adjust class index as needed


                # Draw on black_image

                cvzone.cornerRect(frame, (x, y, w, h), colorR=False, colorC=(0, 0, 0))


                cv2.rectangle(black_image, (x, y), (x + w, y + h), (255, 255, 255), -1)

                # cv2.rectangle(frame, (x, y), (x + w, y + h // 3), (0, 0, 255), 2)
                # cv2.rectangle(frame, (x, y + h // 3), (x + w, y + (h // 3) * 2), (0, 255, 255), 2)
                # cv2.rectangle(frame, (x, y + (h // 3) * 2), (x + w, y + h), (0, 255, 0), 2)


            bitwise = cv2.bitwise_and(frame, black_image)

            # Convert the frame to HSV color space

            hsv_frame = cv2.cvtColor(bitwise, cv2.COLOR_BGR2HSV)


            gray_image = cv2.cvtColor(bitwise, cv2.COLOR_BGR2GRAY)

            # Apply thresholding to the grayscale image
            _, canny_black_image = cv2.threshold(gray_image, 70, 250, cv2.THRESH_BINARY)

            # kernel = np.ones((3, 3), np.uint8)
            #
            # # Apply dilation
            # canny_black_image = cv2.dilate(canny_black_image, kernel)

            if cv2.countNonZero(canny_black_image[y:y + h // 3, x:x + w]) > 20:
                # Show the detected color on the frame
                Traffic_color="Red"
            elif cv2.countNonZero(canny_black_image[y + h // 3:y + (h // 3) * 2, x:x + w]) > 20:
                Traffic_color="Yellow"

            elif cv2.countNonZero(canny_black_image[y + (h // 3) * 2:y + h, x:x + w]) > 20:
                Traffic_color = "Green"

            cv2.putText(frame, f"Traffic Light: {Traffic_color}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 0, 255), 2)


            if  Traffic_color=="Red" and 452 < cy < 455 and 90 < cx < 800:
                # Detect license plates on the cropped region using the plate_model
                vehicle_crop = frame[int(y1):int(y2), int(x1):int(x2)]
                
                # Display the cropped vehicle for visual verification
                #cv2.imshow("Vehicle Crop", vehicle_crop)

                plate_results = plate_model(vehicle_crop, stream=True)

                # Process license plate detection results
                for plate_result in plate_results:
                    plate_boxes = plate_result.boxes

                    for plate_box in plate_boxes:
                        # Get license plate bounding box details
                        px1, py1, px2, py2 = plate_box.xyxy[0]

                        # Crop the detected region (license plate)
                        plate_crop = vehicle_crop[int(py1):int(py2), int(px1):int(px2)]

                        # Apply OCR to the cropped image
                        ocr_result = ocr.ocr(plate_crop, cls=True)
                        if ocr_result and len(ocr_result) > 0:
                            text = ocr_result[0][0][1][0].strip()

                            # Log recognized plate if valid and not processed before
                            if text and len(text) > 4 and text not in processed_numbers:
                                processed_numbers.add(text)
                                current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                print(f"Recognized License Plate: {text} at {current_datetime}")

                                # Optionally, save to a file
                                with open("car_plate_data.txt", "a") as file:
                                    file.write(f"{text}\t{current_datetime}\tRed light violation\n")

                        # Draw a rectangle around the detected license plate
                        cv2.rectangle(vehicle_crop, (int(px1), int(py1)), (int(px2), int(py2)), (0, 255, 0), 2)

    # Draw the traffic light ROI
    cv2.rectangle(frame, (traffic_light_roi[0], traffic_light_roi[1]),
                  (traffic_light_roi[2], traffic_light_roi[3]), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Frame", frame)

    # Press 'Esc' key to exit
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()


[2024/10/01 18:24:54] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\makany/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\makany/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_nu

In [1]:
## Test##


import cvzone
import cv2
import numpy as np
from ultralytics import YOLO
from paddleocr import PaddleOCR
from datetime import datetime
import requests

# Load the YOLO models
plate_model = YOLO("best.pt")  # Model for license plate detection
model = YOLO("yolov8n.pt")  # Model for general object detection

# Initialize PaddleOCR for license plate recognition
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Load the video
cap = cv2.VideoCapture('car.mp4')

# Define the region of interest (ROI) for the traffic light
traffic_light_roi = (900, 100, 960, 160)  # Example values

# To track recognized plates
processed_numbers = set()
po1_rec=(570 ,370)
po12_rec=(176 ,370)
Traffic_color=""
width = 1020
height = 720

# Telegram Bot Token and Chat ID
TOKEN = "7581663246:AAFNYv6F2JRjPkB-5VtIIIO-zue2SVYysAA"

chat_id = "1347052266"

image_path = "violation_licence.jpg"
image_path1 = "violation_car.jpg"


def Send_Image(bot_token, chat_id, image):
    """
    Sends an image to a Telegram chat.

    Parameters:
    - bot_token: The token for the Telegram bot.
    - chat_id: The ID of the Telegram chat to send the image to.
    - image: The file path of the image to send.
    """

    url = f'https://api.telegram.org/bot{bot_token}/sendPhoto'
    files = {'photo': open(image, 'rb')}
    data = {'chat_id': chat_id}
    response = requests.post(url, files=files, data=data)
    return response.json()

def send_image_and_treatment(annotated_image_path, clean_text):
    """
    Send image and treatment information to Telegram.

    Parameters:
    - annotated_image_path: The path to the annotated image.
    - detected_diseases: A list of detected diseases and their treatments.
    """
    Send_Image(TOKEN, chat_id, annotated_image_path)
    
    message = f"this number: {clean_text}"
    url = f'https://api.telegram.org/bot{TOKEN}/sendMessage'
    data = {'chat_id': chat_id, 'text': message}
    requests.post(url, data=data)


while True:
    

    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for consistency
    frame = cv2.resize(frame, (1020, 720))

    # Detect objects (e.g., vehicles) using the general YOLO model
    results = model(frame, stream=True)
    
    black_image = np.zeros((height, width, 3), dtype=np.uint8)
    cv2.line(frame,(po1_rec),(po12_rec),(255, 0, 0),2)

    # Process the detection results from the general YOLO model
    for result in results:
        boxes = result.boxes
        
        for box in boxes:
            # Get the bounding box details
            x1, y1, x2, y2 = box.xyxy[0]
            
            x, y, w, z = int(x1), int(y1), int(x2), int(y2)
            w = w - x
            h = z - y
            cx = x + w // 2
            cy = y + h // 2
            conf = box.conf[0]
            cls = int(box.cls[0])

            if cls == 9:  # Adjust class index as needed


                # Draw on black_image

                cvzone.cornerRect(frame, (x, y, w, h), colorR=False, colorC=(0, 0, 0))


                cv2.rectangle(black_image, (x, y), (x + w, y + h), (255, 255, 255), -1)

                


            bitwise = cv2.bitwise_and(frame, black_image)

            # Convert the frame to HSV color space

            hsv_frame = cv2.cvtColor(bitwise, cv2.COLOR_BGR2HSV)


            gray_image = cv2.cvtColor(bitwise, cv2.COLOR_BGR2GRAY)

            # Apply thresholding to the grayscale image
            _, canny_black_image = cv2.threshold(gray_image, 70, 250, cv2.THRESH_BINARY)

            

            if cv2.countNonZero(canny_black_image[y:y + h // 3, x:x + w]) >20 :
                # Show the detected color on the frame
                Traffic_color="Red"

                
            elif cv2.countNonZero(canny_black_image[y + h // 3:y + (h // 3) * 2, x:x + w]) > 20:
                Traffic_color="Yellow"

            elif cv2.countNonZero(canny_black_image[y + (h // 3) * 2:y + h, x:x + w]) > 20:
                Traffic_color = "Green"

            cv2.putText(frame, f"Traffic Light: {Traffic_color}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 0, 255), 2)


            if  Traffic_color=="Red" and 452 < cy < 455 and 90 < cx < 800:
                
                # Detect license plates on the cropped region using the plate_model
                vehicle_crop = frame[int(y1):int(y2), int(x1):int(x2)]
                cv2.imwrite(image_path1,vehicle_crop )
                
                
                # Display the cropped vehicle for visual verification
                #cv2.imshow("Vehicle Crop", vehicle_crop)

                plate_results = plate_model(vehicle_crop, stream=True)

                # Process license plate detection results
                for plate_result in plate_results:
                    plate_boxes = plate_result.boxes

                    for plate_box in plate_boxes:
                        # Get license plate bounding box details
                        px1, py1, px2, py2 = plate_box.xyxy[0]

                        # Crop the detected region (license plate)
                        plate_crop = vehicle_crop[int(py1):int(py2), int(px1):int(px2)]

                        cv2.imwrite(image_path,plate_crop)


                        # Apply OCR to the cropped image
                        ocr_result = ocr.ocr(plate_crop, cls=True)
                        if ocr_result and len(ocr_result) > 0:
                            text = ocr_result[0][0][1][0].strip()

                            # Log recognized plate if valid and not processed befor

                            if text:
                                text_clean = text.replace(" ", "").strip()
                            else:
                                text_clean = ""
                            
                            # تحقق من صحة النص وتنظيفه قبل المعالجة
                            if text_clean and len(text_clean) > 4 and text_clean not in processed_numbers:
                                processed_numbers.add(text_clean)  # أضف النص النظيفة إلى المجموعة لتتبعها
                                current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                print(f"Recognized License Plate: {text_clean} at {current_datetime}")

                                # حفظ البيانات في الملف بدون فراغات
                                with open("car_plate_data.txt", "a") as file:
                                    Send_Image(TOKEN,chat_id,image_path1)
                                    send_image_and_treatment(image_path, text_clean)
                                    
                                    file.write(f"{text_clean}\t{current_datetime}\tRed light violation\n")


                        # Draw a rectangle around the detected license plate
                        cv2.rectangle(vehicle_crop, (int(px1), int(py1)), (int(px2), int(py2)), (0, 255, 0), 2)

    # Draw the traffic light ROI
    cv2.rectangle(frame, (traffic_light_roi[0], traffic_light_roi[1]),
                  (traffic_light_roi[2], traffic_light_roi[3]), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Frame", frame)

    # Press 'Esc' key to exit
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()


[2024/10/04 02:33:14] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\makany/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\makany/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_nu